In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import requests 
import pandas as pd
import os

client_id = os.environ['SPOTIFY_CID']
client_secret = os.environ['SPOTIFY_CS']

SPOTIPY_REDIRECT_URI = 'http://localhost:65533'
CACHE = '.spotipyoauthcache'
scope = "user-library-read"

from spotipy.oauth2 import SpotifyOAuth

sp2 = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id, client_secret,SPOTIPY_REDIRECT_URI, scope=scope,cache_path=CACHE))

results = []
for i in range(0,1400,50):
    results.append( sp2.current_user_saved_tracks(limit=50,offset=i))
    


In [3]:
track_ids = []
genres = []
track_names = []
album_names = []
artist_names = []

for res in results:
    for item in res['items']:
        track_ids.append(item['track']['id'])
        artist_id = item['track']['album']['artists'][0]['id']
        artist = sp2.artist(artist_id)
        genres.append(artist['genres'])
        track_names.append(item['track']['name'])
        album_names.append(item['track']['album']['name'])
        artist_names.append(item['track']['album']['artists'][0]['name'])
        

In [4]:
df  = pd.DataFrame(list(zip(track_ids, genres,track_names,album_names,artist_names)),columns=['track_ids','genres','track_names','album_names','artist_names'])

In [5]:
df['top_genre'] = ['unknown' if gen==[] else gen[0] for gen in df['genres']]

In [6]:
features = []
for i in range(0,1300,49):
    lis = list((df.loc[i:i+49,'track_ids']))
    features.append(sp2.audio_features(lis))
    
flat_features = [item for sublist in features for item in sublist]
features_dict = { item['id']:item for item in flat_features }

In [7]:
df['danceability'] = [ features_dict[tr_id]['danceability'] for tr_id in df['track_ids'] ]
df['energy'] = [ features_dict[tr_id]['energy'] for tr_id in df['track_ids'] ]
df['key'] = [ features_dict[tr_id]['key'] for tr_id in df['track_ids'] ]
df['loudness'] = [ features_dict[tr_id]['loudness'] for tr_id in df['track_ids'] ]
df['mode'] = [ features_dict[tr_id]['mode'] for tr_id in df['track_ids'] ]
df['speechiness'] = [ features_dict[tr_id]['speechiness'] for tr_id in df['track_ids'] ]
df['acousticness'] = [ features_dict[tr_id]['acousticness'] for tr_id in df['track_ids'] ]
df['instrumentalness'] = [ features_dict[tr_id]['instrumentalness'] for tr_id in df['track_ids'] ]
df['liveness'] = [ features_dict[tr_id]['liveness'] for tr_id in df['track_ids'] ]
df['valence'] = [ features_dict[tr_id]['valence'] for tr_id in df['track_ids'] ]
df['tempo'] = [ features_dict[tr_id]['tempo'] for tr_id in df['track_ids'] ]
df['time_signature'] = [ features_dict[tr_id]['time_signature'] for tr_id in df['track_ids'] ]
df['duration_ms'] = [ features_dict[tr_id]['duration_ms'] for tr_id in df['track_ids'] ]

df.set_index(['track_ids'],inplace=True)

In [8]:
df.shape

(1300, 18)

In [9]:
df.head()

,genres,track_names,album_names,artist_names,top_genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
track_ids,,,,,,,,,,,,,,,,,,
6vaSStNN5NX4nJ4QbRY3S0,[retro soul],Colors,Black Pumas,Black Pumas,retro soul,0.391,0.692,1,-6.543,0,0.0462,0.0946,0.000016,0.1910,0.5600,180.008,4,246587
5YQV81brQo3KZbszwGSKC8,"[contemporary jazz, jazz piano, modern jazz pi...",Markos and Markos,An Ancient Observer,Tigran Hamasyan,contemporary jazz,0.434,0.115,8,-20.806,0,0.0355,0.9890,0.874000,0.1120,0.0816,86.820,4,338200
6I7GNd3UW45TAxG49R4HtX,[tamil pop],Megathoodham,Airaa (Original Motion Picture Soundtrack),Sundaramurthy K.S.,tamil pop,0.706,0.427,4,-10.191,0,0.0416,0.3470,0.000004,0.1080,0.4180,129.942,1,267292
2K8elWg8ihrZRwZJ7Gy6L3,"[dance pop, indie cafe pop, indie pop, indie p...",Come Home To Me,LÉON,LÉON,dance pop,0.660,0.193,11,-8.904,1,0.0725,0.9530,0.000000,0.0953,0.2960,82.035,4,225868
018eOid2aGaPdxon7T6GsC,"[desi hip hop, desi pop, filmi, indian instrum...","Shayad (From ""Love Aaj Kal"")","Shayad (From ""Love Aaj Kal"")",Pritam,desi hip hop,0.487,0.456,8,-7.144,1,0.0319,0.7890,0.000002,0.1410,0.3480,136.088,4,247941


In [10]:
df

,genres,track_names,album_names,artist_names,top_genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
track_ids,,,,,,,,,,,,,,,,,,
6vaSStNN5NX4nJ4QbRY3S0,[retro soul],Colors,Black Pumas,Black Pumas,retro soul,0.391,0.6920,1,-6.543,0,0.0462,0.0946,0.000016,0.1910,0.5600,180.008,4,246587
5YQV81brQo3KZbszwGSKC8,"[contemporary jazz, jazz piano, modern jazz pi...",Markos and Markos,An Ancient Observer,Tigran Hamasyan,contemporary jazz,0.434,0.1150,8,-20.806,0,0.0355,0.9890,0.874000,0.1120,0.0816,86.820,4,338200
6I7GNd3UW45TAxG49R4HtX,[tamil pop],Megathoodham,Airaa (Original Motion Picture Soundtrack),Sundaramurthy K.S.,tamil pop,0.706,0.4270,4,-10.191,0,0.0416,0.3470,0.000004,0.1080,0.4180,129.942,1,267292
2K8elWg8ihrZRwZJ7Gy6L3,"[dance pop, indie cafe pop, indie pop, indie p...",Come Home To Me,LÉON,LÉON,dance pop,0.660,0.1930,11,-8.904,1,0.0725,0.9530,0.000000,0.0953,0.2960,82.035,4,225868
018eOid2aGaPdxon7T6GsC,"[desi hip hop, desi pop, filmi, indian instrum...","Shayad (From ""Love Aaj Kal"")","Shayad (From ""Love Aaj Kal"")",Pritam,desi hip hop,0.487,0.4560,8,-7.144,1,0.0319,0.7890,0.000002,0.1410,0.3480,136.088,4,247941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4aHJpZ6AMXOQAEMNWBUYfw,[],The Wish - Live / Instrumental,Remember Shakti,Various Artists,unknown,0.432,0.5620,0,-14.042,1,0.1380,0.7760,0.830000,0.6790,0.1320,160.854,3,1128000
4cAZTObjvJoMGafY6C5XWn,[],Lotus Feet - Live / Instrumental,Remember Shakti,Various Artists,unknown,0.583,0.2770,0,-15.876,1,0.0379,0.9250,0.915000,0.6860,0.1550,110.073,3,441067
79egwGUyFpkUP52K0vE3ab,[],Mukti - Live / Instrumental,Remember Shakti,Various Artists,unknown,0.505,0.4260,11,-18.799,1,0.1470,0.5760,0.842000,0.6570,0.3200,87.863,4,3811360


In [13]:
df.to_csv("my_spotify_data.csv")